In [1]:
# Escribe un requirements.txt y lo instala en ESTE kernel
req = """\
osmnx==1.9.3
networkx==3.3
matplotlib==3.9.2
scikit-learn==1.5.2
rtree==1.3.0
shapely==2.0.6
geopandas==1.0.1
pyproj==3.7.0
fiona==1.10.1
"""
with open("requirements.txt", "w") as f:
    f.write(req)

import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
print("✅ Dependencias instaladas.")

✅ Dependencias instaladas.


## Ejercicio 1
Completa los algoritmos de **Bellman–Ford**, **Kruskal** y **Prim** basándote en la bibliografía indicada (Usa Dijkstra como referencia):  
- *The Algorithm Design Manual*, Steven Skiena (2008).  

Implementa cada uno en Python siguiendo las referencias, y compara sus resultados sobre un mismo grafo.


In [ ]:
from typing import Any, Dict, List, Tuple
import heapq, math

Graph = Dict[Any, List[Tuple[Any, float]]]

def dijkstra(graph: Graph, source: Any):
    nodes = set(graph.keys())
    for u in graph:
        for v, _ in graph[u]:
            nodes.add(v)
    dist = {n: math.inf for n in nodes}
    parent = {source: None}
    dist[source] = 0.0
    pq = [(0.0, source)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]:
            continue
        for v, w in graph.get(u, []):
            if w < 0:
                raise ValueError("Dijkstra requiere pesos no negativos.")
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                parent[v] = u
                heapq.heappush(pq, (nd, v))
    return dist, parent

def reconstruct_path(parent: Dict[Any, Any], s: Any, t: Any) -> List[Any]:
    if t not in parent and t != s:
        return []
    path, cur = [], t
    while cur is not None:
        path.append(cur)
        cur = parent.get(cur, None)
    if path[-1] != s:
        return []
    path.reverse()
    return path

# Grafo pequeño dirigido y ponderado (pesos >= 0)
G1: Graph = {
    'A': [('B', 2), ('C', 5)],
    'B': [('C', 1), ('D', 3)],
    'C': [('D', 2)],
    'D': []
}

dist, parent = dijkstra(G1, 'A')
print("Dijkstra dist:", dist)
print("Dijkstra camino A->D:", reconstruct_path(parent, 'A', 'D'))

Dijkstra dist: {'B': 2.0, 'C': 3.0, 'A': 0.0, 'D': 5.0}
Dijkstra camino A->D: ['A', 'B', 'D']


In [ ]:
from typing import Any, Dict, List, Tuple
import math

Graph = Dict[Any, List[Tuple[Any, float]]]

def bellman_ford(graph: Graph, source: Any):
    edges = [(u, v, w) for u in graph for (v, w) in graph[u]]
    nodes = set(graph.keys())
    for u, v, _ in edges:
        nodes.add(u); nodes.add(v)
    dist = {n: math.inf for n in nodes}
    parent = {n: None for n in nodes}
    dist[source] = 0.0
    # Relajar V-1 veces
    for _ in range(len(nodes) - 1):
        updated = False
            # TO DO
        for u, v, w in edges:
            if dist[u] != math.inf and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
            parent[v] = u
            updated = True

    # Detección de ciclo negativo
    has_neg_cycle = any(dist[u] != math.inf and dist[u] + w < dist[v] for u, v, w in edges)
    return dist, parent, has_neg_cycle

def reconstruct_path(parent: Dict[Any, Any], s: Any, t: Any) -> List[Any]:
    if t not in parent and t != s:
        return []
    path, cur = [], t
    while cur is not None:
        path.append(cur)
        cur = parent.get(cur, None)
    if path[-1] != s:
        return []
    path.reverse()
    return path

# Grafo (permite pesos negativos, sin ciclo negativo)
G2: Graph = {
    'S': [('A', 4), ('B', 5)],
    'A': [('C', -2)],
    'B': [('C', 3)],
    'C': [('T', 2)],
    'T': []
}
dist, parent, neg = bellman_ford(G2, 'S')
print("Bellman-Ford dist:", dist, "neg_cycle:", neg)
print("Bellman-Ford camino S->T:", reconstruct_path(parent, 'S', 'T'))


# Ref:
# Ciclos de coste negativo hacen indefinido el problema: p. 491 (y mención de “negative-cost cycle” p. 490) .
# Índice de “Bellman-Ford algorithm”: p. 490, 493 .

Bellman-Ford dist: {'S': 0.0, 'C': 2.0, 'A': 4.0, 'B': 5.0, 'T': 4.0} neg_cycle: False
Bellman-Ford camino S->T: ['S', 'B', 'C', 'T']


In [ ]:
from typing import Any, Dict, List, Tuple
import heapq

UGraph = Dict[Any, List[Tuple[Any, float]]]

def prim_mst(graph: UGraph, start=None):
    if not graph:
        return 0.0, []
    if start is None:
        start = next(iter(graph))
    visited = {start}
    pq: List[Tuple[float, Any, Any]] = []
    for v, w in graph[start]:
        heapq.heappush(pq, (w, start, v))
    mst, total = [], 0.0
    while pq and len(visited) < len(graph):
        w, u, v = heapq.heappop(pq)
        #TO DO
        if v in visited:           # ya está en el árbol: descartar
            continue
        visited.add(v)             # aceptamos la arista (u,v)
        mst.append((u, v, w))
        total += w
        for x, wx in graph.get(v, []):
            if x not in visited:
                heapq.heappush(pq, (wx, v, x))
    return total, mst

# Grafo no dirigido (simétrico)
G3: UGraph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('A', 1), ('C', 2), ('D', 5)],
    'C': [('A', 4), ('B', 2), ('D', 1)],
    'D': [('B', 5), ('C', 1)]
}
total, edges = prim_mst(G3, 'A')
print("Prim MST peso total:", total)
print("Prim MST aristas:", edges)

# Ref:
# Greedy, añadir arista mínima árbol↔no-árbol, no crea ciclos: Skiena, p. 193–194 .
# Prioridad/heap hasta O(m + n log n): Skiena, p. 196 .
# Cuándo preferir Prim o Kruskal según densidad: Skiena, p. 487 .

Prim MST peso total: 4.0
Prim MST aristas: [('A', 'B', 1), ('B', 'C', 2), ('C', 'D', 1)]


In [ ]:
from typing import Any, Dict, List, Tuple

UGraph = Dict[Any, List[Tuple[Any, float]]]

class DSU:
    def __init__(self, items):
        self.parent = {x: x for x in items}
        self.rank = {x: 0 for x in items}
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb: return False
        if self.rank[ra] < self.rank[rb]:
            ra, rb = rb, ra
        self.parent[rb] = ra
        if self.rank[ra] == self.rank[rb]:
            self.rank[ra] += 1
        return True

def kruskal_mst(graph: UGraph):
    verts = set(graph.keys())
    edges = []
    seen = set()
    for u in graph:
       #TO DO
       for v, w in graph[u]:
            # incluir vertices que solo aparecen como vecinos
            verts.add(u); verts.add(v)
            if u == v:
                continue  # opcional: ignorar bucles
            a, b = (u, v) if u <= v else (v, u)  # ordenar extremos para no duplicar
            if (a, b) in seen:
                continue
            seen.add((a, b))
            edges.append((w, a, b))
    edges.sort()
    dsu = DSU(verts)
    mst, total = [], 0.0
    for w, u, v in edges:
        if dsu.union(u, v):
            mst.append((u, v, w))
            total += w
        if len(mst) == len(verts) - 1:
            break
    return total, mst

#  Grafo no dirigido
G4: UGraph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('A', 1), ('C', 2), ('D', 5)],
    'C': [('A', 4), ('B', 2), ('D', 1)],
    'D': [('B', 5), ('C', 1)]
}
total, edges = kruskal_mst(G4)
print("Kruskal MST peso total:", total)
print("Kruskal MST aristas:", edges)

# Notas:
# Kruskal también es greedy, pero ordena aristas y usa DSU para evitar
# El punto es ordenar aristas por peso e ir uniendo componentes si la arista no crea ciclo; 
# termina con n−1 aristas (Skiena, p. 197–198).
# Union-Find: union by size/rank y path compression reducen la altura casi a constante (Skiena, p. 388).
# Corrección (intuición del intercambio): si una arista “ligera” conectó dos componentes, cualquier arista posterior en el ciclo sería 
# >= su peso; intercambiar no empeora (Skiena, p. 197–198).

Kruskal MST peso total: 4.0
Kruskal MST aristas: [('A', 'B', 1), ('C', 'D', 1), ('B', 'C', 2)]


## Ejercicio 2
En el siguiente ejercicio se puede apreciar un **mapa de Montevideo** con su ruta más al **este** y su ruta más al **oeste**.  

1. Corre los algoritmos para calcular 
2. Observa y responde:  
   a) ¿Cuál es más eficiente y por qué?  
   b) ¿En qué casos ambos algoritmos devuelven resultados idénticos?  
   c) Explica las diferencias en términos de complejidad y aplicabilidad.

In [2]:
import os
import time
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

ox.settings.use_cache = True
ox.settings.log_console = True

GRAPHML_PATH = "montevideo_drive.graphml"

def draw_edge(ax, G, u, v, color="orange", lw=1.2, alpha=0.9):
    """Dibuja (u,v) buscando ambas direcciones en el MultiDiGraph; si no hay geometría, usa segmento recto."""
    data = G.get_edge_data(u, v, default=None)
    if data is None:
        data = G.get_edge_data(v, u, default=None)
        if data is None:
            return
        # v->u
        key = min(data.keys(), key=lambda k: data[k].get("length", float("inf")))
        geom = data[key].get("geometry", None)
        if geom is not None:
            xs, ys = geom.xy
        else:
            xs = [G.nodes[v]["x"], G.nodes[u]["x"]]
            ys = [G.nodes[v]["y"], G.nodes[u]["y"]]
    else:
        # u->v
        key = min(data.keys(), key=lambda k: data[k].get("length", float("inf")))
        geom = data[key].get("geometry", None)
        if geom is not None:
            xs, ys = geom.xy
        else:
            xs = [G.nodes[u]["x"], G.nodes[v]["x"]]
            ys = [G.nodes[u]["y"], G.nodes[v]["y"]]
    ax.plot(xs, ys, linewidth=lw, color=color, alpha=alpha)

def draw_route(ax, G, route, color="red", lw=3.0, alpha=0.95):
    if not route or len(route) < 2:
        return
    for u, v in zip(route[:-1], route[1:]):
        draw_edge(ax, G, u, v, color=color, lw=lw, alpha=alpha)

if os.path.exists(GRAPHML_PATH):
    G_wgs = ox.load_graphml(GRAPHML_PATH)
    print(f"Cargado desde cache: {GRAPHML_PATH}")
else:
    print("Descargando red vial de Montevideo (drive)…")
    G_wgs = ox.graph_from_place("Montevideo, Uruguay", network_type="drive", simplify=True)
    ox.save_graphml(G_wgs, GRAPHML_PATH)
    print(f"Guardado cache: {GRAPHML_PATH}")

G = ox.project_graph(G_wgs)

xs = {n: G.nodes[n]["x"] for n in G.nodes}
orig_node = min(xs, key=xs.get)   # extremo oeste
dest_node = max(xs, key=xs.get)   # extremo este
print("Nodos elegidos (lejanos):", orig_node, "->", dest_node)

#  Convertir G a diccionarios compatibles conb la libreria
# (1) Grafo DIRIGIDO para SSSP: dict[u] -> [(v, w_min)]
directed_weights = {}  # (u,v) -> min length
for u, v, k, data in G.edges(keys=True, data=True):
    w = float(data.get("length", 1.0))
    key = (u, v)
    if key not in directed_weights or w < directed_weights[key]:
        directed_weights[key] = w

graph_dir = {}
for (u, v), w in directed_weights.items():
    graph_dir.setdefault(u, []).append((v, w))
    if u not in graph_dir:
        graph_dir[u] = graph_dir.get(u, [])
    if v not in graph_dir:
        graph_dir.setdefault(v, [])

# (2) Grafo NO DIRIGIDO "simple" para MST: dict[u] -> [(v, w_min)] y simétrico
undirected_weights = {}  # {frozenset({u,v}): w_min}
for u, v, k, data in G.edges(keys=True, data=True):
    w = float(data.get("length", 1.0))
    key = frozenset((u, v))
    if u == v:
        continue
    if key not in undirected_weights or w < undirected_weights[key]:
        undirected_weights[key] = w

graph_undir = {}
for key, w in undirected_weights.items():
    u, v = tuple(key)
    graph_undir.setdefault(u, []).append((v, w))
    graph_undir.setdefault(v, []).append((u, w))

# --------- Ejecución de algoritmos ---------
# Dijkstra 
t0 = time.perf_counter() 
dist_dij, parent_dij = dijkstra(graph_dir, orig_node) 
t1 = time.perf_counter() 
path_dij = reconstruct_path(parent_dij, orig_node, dest_node) 
t_dij = t1 - t0 

# Bellman–Ford 
t0 = time.perf_counter() 
dist_bf, parent_bf, neg_cycle = bellman_ford(graph_dir, orig_node) 
t1 = time.perf_counter() 
path_bf = reconstruct_path(parent_bf, orig_node, dest_node) 
t_bf = t1 - t0

# Prim (MST) — opcionalmente podés fijar start=orig_node
t0 = time.perf_counter()
total_prim, mst_edges_prim = prim_mst(graph_undir, start=orig_node)
t1 = time.perf_counter()
t_prim = t1 - t0

# Kruskal (MST)
t0 = time.perf_counter()
total_kr, mst_edges_kr = kruskal_mst(graph_undir)
t1 = time.perf_counter()
t_kr = t1 - t0

#  Camino dentro de cada MST entre orig_node y dest_node 
def path_in_mst_edges(edges, s, t):
    """BFS sobre las aristas del MST (no dirigido) para recuperar el camino entre s y t."""
    adj = {}
    for u, v, w in edges:
        adj.setdefault(u, []).append(v)
        adj.setdefault(v, []).append(u)
    if s not in adj or t not in adj:
        return []
    from collections import deque
    q = deque([s])
    prev = {s: None}
    while q:
        u = q.popleft()
        if u == t:
            break
        for v in adj.get(u, []):
            if v not in prev:
                prev[v] = u
                q.append(v)
    if t not in prev:
        return []
    # reconstruir
    path = []
    cur = t
    while cur is not None:
        path.append(cur)
        cur = prev[cur]
    return list(reversed(path))

path_mst_prim   = path_in_mst_edges(mst_edges_prim, orig_node, dest_node)
path_mst_krusky = path_in_mst_edges(mst_edges_kr,  orig_node, dest_node)

# --------- Reporte de tiempos ---------
print(f"Dijkstra       : {t_dij:.3f} s — nodos en ruta: {len(path_dij)}")
print(f"Bellman–Ford   : {t_bf:.3f} s — nodos en ruta: {len(path_bf)} — ciclo negativo: {neg_cycle}")
print(f"Prim (MST)     : {t_prim:.3f} s — aristas MST: {len(mst_edges_prim)} — peso: {total_prim:.1f}")
print(f"Kruskal (MST)  : {t_kr:.3f} s — aristas MST: {len(mst_edges_kr)} — peso: {total_kr:.1f}")

# --------- Plot final (una ruta por algoritmo) ---------
fig, ax = ox.plot_graph(
    G, bgcolor="white", node_size=0,
    edge_color="lightgray", edge_linewidth=0.4,
    show=False, close=False, figsize=(10, 10)
)

# Caminos SSSP (tus rutas)
draw_route(ax, G, path_dij,        color="red",    lw=3.0, alpha=0.95)
draw_route(ax, G, path_bf,         color="blue",   lw=2.4, alpha=0.95)
# Caminos dentro de los MST
draw_route(ax, G, path_mst_prim,   color="orange", lw=2.0, alpha=0.9)
draw_route(ax, G, path_mst_krusky, color="purple", lw=1.6, alpha=0.9)

# Marcas de origen/destino
ax.scatter(G.nodes[orig_node]["x"], G.nodes[orig_node]["y"], c="black", s=36, zorder=5)
ax.scatter(G.nodes[dest_node]["x"], G.nodes[dest_node]["y"], c="black", s=36, zorder=5)

legend = [
    Line2D([0], [0], color="red",    lw=3.0, label="Dijkstra "),
    Line2D([0], [0], color="blue",   lw=2.4, label="Bellman–Ford "),
    Line2D([0], [0], color="orange", lw=2.0, label="MST Prim "),
    Line2D([0], [0], color="purple", lw=1.6, label="MST Kruskal "),
]
ax.legend(handles=legend, loc="lower left")
ax.set_title("Montevideo — Tus algoritmos + tiempos de ejecución", fontsize=13)
plt.show()


Cargado desde cache: montevideo_drive.graphml
Nodos elegidos (lejanos): 8907760088 -> 917534855


NameError: name 'dijkstra' is not defined

a) ¿Cuál es más eficiente y por qué?

- En una red vial como la de OSM (longitudes siempre ≥ 0), Dijkstra es el indicado y más eficiente: funciona cuando no hay aristas de costo negativo, y su implementación moderna con colas de prioridad/Fibonacci heaps alcanza 
O(m+nlogn) (cap. 15.4, págs. 491–494). También se muestra la versión 
O(n2) “simple” (misma que Prim con arrays) (cap. 6.3, p. 209).
- Bellman-Ford se usa cuando hay pesos negativos (o para detectar ciclos negativos), pero es “más general, aunque menos eficiente” (cap. 15.4, p. 491).
Por ello, Dijkstra será más rápido que Bellman-Ford.

b) ¿Cuándo devuelven resultados idénticos?

Cuando todas las aristas tienen peso no negativo (como en la red vial) y no hay ciclos negativos. En ese caso, ambos calculan las mismas distancias/rutas de camino mínimo desde el mismo origen.

c) Diferencias en complejidad y aplicabilidad (incluyo MST para contrastar con tus dos rutas “MST”):
- Dijkstra (SSSP con pesos ≥ 0): O(m+nlogn) con colas/estructuras avanzadas. Uso típico: ruteo punto-a-punto en redes viales (cap. 5, p. 146; cap. 15.4, p. 494).
- Bellman-Ford (SSSP con pesos negativos y detección de ciclos): más general pero menos eficiente. Skiena no da la cota explícita en ese párrafo; sin embargo, la cota estándar clásica es O(nm).
- Prim (MST, conecta todos los vértices minimizando el peso total): versión básica O(n2); con prioridad O(m+nlogn).
- Kruskal (MST): O(mlogm) con union-find.


NOTA: ¿Prim o Kruscal? Prim (simple) es mejor en grafos densos y Kruskal en dispersos; con estructuras avanzadas, un Prim con heaps empareja o supera en la práctica (cap. 15.3, p. 487).

## Ejercicio 3
En este ejercicio se corren **N veces los algoritmos** sobre el mapa de Montevideo con **direcciones aleatorias**.  

Analiza:  
a) ¿Cuál es el algoritmo más eficiente en promedio?  
b) ¿Cuál demora más y por qué motivo?  
c) ¿Cuál utilizarías tú en un sistema en producción y por qué?

In [5]:
# ✅ Celda nueva — Benchmark de tiempos promedio en Montevideo (n=10)
import random
import time

def benchmark_algorithms(G, graph_dir, graph_undir, n=10):
    times_dij, times_bf, times_prim, times_kr = [], [], [], []

    # Convertimos explícitamente los nodos a lista
    nodes = list(G.nodes())

    for _ in range(n):
        # elegimos dos nodos distintos al azar dentro del dominio del grafo
        orig, dest = random.sample(nodes, 2)

        # Dijkstra
        t0 = time.perf_counter()
        dist_dij, parent_dij = dijkstra(graph_dir, orig)
        path_dij = reconstruct_path(parent_dij, orig, dest)
        t1 = time.perf_counter()
        times_dij.append(t1 - t0)

        # Bellman-Ford
        t0 = time.perf_counter()
        dist_bf, parent_bf, neg_cycle = bellman_ford(graph_dir, orig)
        path_bf = reconstruct_path(parent_bf, orig, dest)
        t1 = time.perf_counter()
        times_bf.append(t1 - t0)

        # Prim
        t0 = time.perf_counter()
        total_prim, mst_edges_prim = prim_mst(graph_undir, start=orig)
        t1 = time.perf_counter()
        times_prim.append(t1 - t0)

        # Kruskal
        t0 = time.perf_counter()
        total_kr, mst_edges_kr = kruskal_mst(graph_undir)
        t1 = time.perf_counter()
        times_kr.append(t1 - t0)

    # Promedios
    print(f"Promedio Dijkstra     : {sum(times_dij)/n:.4f} s")
    print(f"Promedio Bellman–Ford : {sum(times_bf)/n:.4f} s")
    print(f"Promedio Prim (MST)   : {sum(times_prim)/n:.4f} s")
    print(f"Promedio Kruskal (MST): {sum(times_kr)/n:.4f} s")

benchmark_algorithms(G, graph_dir, graph_undir, n=50)

NameError: name 'dijkstra' is not defined

a) ¿Cuál es el algoritmo más eficiente en promedio?

Para rutas s-t (SSSP): Dijkstra es el más eficiente (las longitudes de calles son no negativas). Skiena enfatiza que con pesos no negativos Dijkstra es el método indicado y que Bellman–Ford se reserva para el caso más general (con pesos negativos), pero es menos eficiente, p. 491.

Para el MST del grafo: Kruskal tiende a ganar en grafos dispersos (como redes viales) con costo O(m log m), mientras que Prim “básico” es O(n2), p. 487. Con una cola de prioridad buena, Prim baja a O(m+n log n) y suele ser la opción práctica más rápida en la mayoría de los grafos, p. 196; , p. 487.

b) ¿Cuál demora más y por qué motivo?

Bellman–Ford demora más: es “más general pero menos eficiente” que Dijkstra, p. 491. La razón es que relaja todas las aristas repetidas veces (V−1 rondas) y además debe chequear ciclos de costo negativo; si existiera un ciclo negativo, las distancias ni siquiera están bien definidas, p. 491, eso no aporta ventaja.

Entre Prim y Kruskal: con implementaciones del texto, Prim (O(n2)) suele tardar más en grafos grandes y ralos; Kruskal (O(m log m)) con union–find eficiente es mejor en esos casos, p. 197–198; , p. 485. Si Prim usa una buena cola de prioridad, ambos quedan en la misma liga y Prim suele ser el “caballo de batalla” práctico, p. 487.

c) ¿Cuál usaría en producción y por qué?

Para ruteo urbano (muchas consultas s-t): Dijkstra con cola de prioridad (o A* con heurística si querés acelerar más). Es correcto con pesos no negativos y muy eficiente, pp. 374–376.

Para construir una sola vez el MST de la red (si es para análisis/clusterización, no para ruteo): Prim con cola de prioridad o Kruskal con union–find. En redes viales (dispersas), Kruskal es simple y sólido; si ya hay una buena priority queue, Prim (O(m+n log n)) es excelente, p. 196; , p. 487.